In [1]:
# импортируем необходимые библиотеки
import pandas as pd
import numpy as np
# загружаем данные
tr = pd.read_csv('Data/Stat_FE.csv', encoding='cp1251', sep=';')
# выводим наблюдения
tr

,Class,Response
0,A,1
1,A,0
2,A,1
3,A,1
4,B,1
5,B,1
6,B,0
7,C,1
8,C,1


## Дамми-кодирование (One-Hot Encoding)

### Дамми-кодирование с помощью функции get_dummies() библиотеки pandas

In [2]:
# выполняем дамми-кодирование 
# по методу неполного ранга
dummies_unfull_rank_class = pd.get_dummies(tr['Class'], 
                                           drop_first=False)
# выводим наблюдения
dummies_unfull_rank_class

,A,B,C
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,0,1,0
5,0,1,0
6,0,1,0
7,0,0,1
8,0,0,1


### Дамми-кодирование по методу полного ранга

In [3]:
# выполняем дамми-кодирование 
# по методу полного ранга
dummies_full_rank_class = pd.get_dummies(tr['Class'], 
                                         drop_first=True)
# выводим наблюдения
dummies_full_rank_class

,B,C
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
5,1,0
6,1,0
7,0,1
8,0,1


### Дамми-кодирование с помощью класса OneHotEncoder

In [4]:
# импортируем класс OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
# создаем экземпляр класса OneHotEncoder
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
# обучаем модель дамми-кодирования - определяем 
# дамми для переменной Class
ohe.fit(tr['Class'].values.reshape(-1, 1))
# выполняем дамми-кодирование переменной Class по методу 
# неполного ранга в обучающем массиве признаков
ohe_train_unfull_rank = ohe.transform(tr['Class'].values.reshape(-1, 1))
ohe_train_unfull_rank

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

## Кодирование контрастами (Effect Encoding)

In [5]:
# выполняем кодирование контрастами
effects_class = pd.get_dummies(tr['Class'])
effects_class = effects_class.iloc[:,:-1]
effects_class.loc[np.all(effects_class == 0, axis=1)] = -1.

# выводим наблюдения
effects_class

,A,B
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0
5,0.0,1.0
6,0.0,1.0
7,-1.0,-1.0
8,-1.0,-1.0


## Присвоение категориям в лексикографическом порядке целочисленных значений, начиная с 0 (Label Encoding)

In [6]:
# импортируем класс LabelEncoder
from sklearn.preprocessing import LabelEncoder
# создаем экземпляр класса LabelEncoder
label_encoder = LabelEncoder().fit(tr['Class'])
# выполняем кодировку
tr['class_labelenc'] = label_encoder.transform(tr['Class'])
tr

,Class,Response,class_labelenc
0,A,1,0
1,A,0,0
2,A,1,0
3,A,1,0
4,B,1,1
5,B,1,1
6,B,0,1
7,C,1,2
8,C,1,2


## Кодирование частотами (Frequency Encoding)

In [7]:
# создаем переменную class_abs_freq, у которой каждое значение - абсолютная 
# частота наблюдений в категории переменной Class
abs_freq = tr['Class'].value_counts()
tr['class_abs_freq'] = tr['Class'].map(abs_freq)
tr

,Class,Response,class_labelenc,class_abs_freq
0,A,1,0,4
1,A,0,0,4
2,A,1,0,4
3,A,1,0,4
4,B,1,1,3
5,B,1,1,3
6,B,0,1,3
7,C,1,2,2
8,C,1,2,2


In [8]:
# создаем переменную class_rel_freq, у которой каждое значение - относительная 
# частота наблюдений в категории переменной Class
rel_freq = tr['Class'].value_counts() / len(tr['Class'])
tr['class_rel_freq'] = tr['Class'].map(rel_freq)
tr

,Class,Response,class_labelenc,class_abs_freq,class_rel_freq
0,A,1,0,4,0.444444
1,A,0,0,4,0.444444
2,A,1,0,4,0.444444
3,A,1,0,4,0.444444
4,B,1,1,3,0.333333
5,B,1,1,3,0.333333
6,B,0,1,3,0.333333
7,C,1,2,2,0.222222
8,C,1,2,2,0.222222


## Кодирование вероятностями (Likelihood Encoding)

### Кодирование простым средним значением зависимой переменной

In [9]:
def simple_mean_target_encoding(df, target, column):
    mean_enc = df.groupby(column)[target].mean()
    df['mean_encoded'] = df[column].map(mean_enc)
    return df
tr = simple_mean_target_encoding(tr, 'Response', 'Class')
tr

,Class,Response,class_labelenc,class_abs_freq,class_rel_freq,mean_encoded
0,A,1,0,4,0.444444,0.750000
1,A,0,0,4,0.444444,0.750000
2,A,1,0,4,0.444444,0.750000
3,A,1,0,4,0.444444,0.750000
4,B,1,1,3,0.333333,0.666667
5,B,1,1,3,0.333333,0.666667
6,B,0,1,3,0.333333,0.666667
7,C,1,2,2,0.222222,1.000000
8,C,1,2,2,0.222222,1.000000


### Кодирование простым средним значением зависимой переменной по схеме K-fold

In [10]:
# загружаем данные
train = pd.read_csv('Data/Stat_FE2_train.csv', sep=';')
test = pd.read_csv('Data/Stat_FE2_test.csv', sep=';')

In [11]:
# выводим наблюдения обучающего набора
train

,living_region,job_position,open_account_flg
0,Московская область,Служащий,0
1,Московская область,Заместитель руководителя,1
2,Пермский край,Служащий,1
3,Московская область,Руководитель,0
4,Пермский край,Руководитель,1
5,Пермский край,Руководитель,1
6,Пермский край,Руководитель,1
7,Пермский край,Руководитель,0
8,Московская область,Заместитель руководителя,1
9,Московская область,Заместитель руководителя,0


In [12]:
# выводим наблюдения тестового набора
test

,living_region,job_position,open_account_flg
0,Пермский край,Служащий,0
1,Московская область,Заместитель руководителя,0
2,Пермский край,Служащий,1
3,Московская область,Руководитель,0
4,Пермский край,Служащий,1
5,Пермский край,Руководитель,0
6,Пермский край,Руководитель,1
7,Московская область,Заместитель руководителя,1
8,Московская область,Заместитель руководителя,1
9,Свердловская область,Заместитель руководителя,0


In [13]:
# импортируем класс StratifiedKFold
from sklearn.model_selection import StratifiedKFold
# создаем экземпляр класса StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
# создаем список из двух признаков и зависимой переменной
cat_cols = ['living_region', 'job_position', 'open_account_flg']
# создаем обучающий массив со значениями зависимой переменной
y_train = train.loc[:, 'open_account_flg'].astype('int')
# создаем обучающий массив признаков
X_train = train[cat_cols].drop('open_account_flg', axis=1)
# создаем тестовый массив признаков
X_test = test[cat_cols].drop('open_account_flg', axis=1)

In [15]:
# пишем функцию, выполняющую кодирование средними
# значениями зависимой переменной
def mean_target_enc(X_train, y_train, X_valid, skf):
    # отключаем предупреждения Anaconda
    import warnings
    warnings.filterwarnings('ignore')
    
    # вычисляем глобальное среднее - среднее значение 
    # зависимой переменной в обучающем наборе
    glob_mean = y_train.mean()
    
    # конкатенируем обучающий массив с признаками (задается первым аргументом) 
    # и обучающий массив с метками зависимой переменной (задается вторым
    # аргументом) по оси столбцов 
    X_train = pd.concat([X_train, pd.Series(y_train, name='open_account_flg')], axis=1)
    # создаем копию массива признаков, получившегося в результате конкатенации
    new_X_train = X_train.copy()
    
    # создаем список с именами категориальных признаков,
    # который мы будем использовать ниже в циклах for
    cat_features = X_train.columns[X_train.dtypes == 'object'].tolist()
    
    # для каждого категориального признака создаем столбец, каждое
    # значение которого - глобальное среднее
    for col in cat_features:
        new_X_train[col + '_mean_target'] = [glob_mean for _ in range(new_X_train.shape[0])]

    # вычисляем среднее значение зависимой переменной в категории признака
    # по каждому блоку перекрестной проверки, используя данные вне этого блока
    # например, мы используем 5-блочную перекрестную проверку и нам нужно 
    # вычислить среднее значение зависимой переменной для категории A в блоке 0,
    # для вычисления этого среднего значения используются лишь наблюдения в категории А 
    # в обучающих блоках 1, 2, 3, 4, если вместо категорий у нас 
    # значения NaN, заменяем глобальным средним, в итоге
    # получаем новый обучающий набор
    for train_idx, valid_idx in skf.split(X_train, y_train):
        X_train_cv, X_valid_cv = X_train.iloc[train_idx, :], X_train.iloc[valid_idx, :]

        for col in cat_features:            
            means = X_valid_cv[col].map(X_train_cv.groupby(col)['open_account_flg'].mean())            
            X_valid_cv[col + '_mean_target'] = means.fillna(glob_mean)
            
        new_X_train.iloc[valid_idx] = X_valid_cv
    
    # удаляем из нового обучающего набора категориальные признаки и зависимую переменную
    new_X_train.drop(cat_features + ['open_account_flg'], axis=1, inplace=True)
    
    # создаем копию тестового массива признаков
    new_X_valid = X_valid.copy()
    
    # каждую категорию категориального признака в тестовом наборе
    # заменяем средним значением зависимой переменной в этой же категории
    # признака, вычисленным на обучающем наборе, значения NaN 
    # заменяем глобальным средним
    for col in cat_features:        
        means = new_X_valid[col].map(X_train.groupby(col)['open_account_flg'].mean())        
        new_X_valid[col + '_mean_target'] = means.fillna(glob_mean)
    
    # удаляем из тестового набора категориальные признаки
    new_X_valid.drop(X_train.columns[X_train.dtypes == 'object'], axis=1, inplace=True)
    
    # возвращаем новые датафреймы
    return new_X_train, new_X_valid

In [16]:
# выполняем кодирование средними значениями
# зависимой переменной для переменных
# living_region и job_position
# в обучающем и тестовом наборах
train_mean_target, test_mean_target = mean_target_enc(X_train, y_train, X_test, skf)

In [17]:
# взглянем на результаты кодировки в обучающем наборе
train_mean_target

,living_region_mean_target,job_position_mean_target
0,0.333333,0.500000
1,0.333333,0.400000
2,0.750000,0.000000
3,0.333333,0.750000
4,0.750000,0.500000
5,0.750000,0.500000
6,1.000000,0.666667
7,1.000000,0.666667
8,0.333333,0.400000
9,0.500000,0.600000


In [18]:
# взглянем на результаты кодировки в тестовом наборе
test_mean_target

,living_region_mean_target,job_position_mean_target
0,0.8,0.25
1,0.4,0.50
2,0.8,0.25
3,0.4,0.60
4,0.8,0.25
5,0.8,0.60
6,0.8,0.60
7,0.4,0.50
8,0.4,0.50
9,0.2,0.50


### Кодирование средним значением зависимой переменной,  сглаженным через сигмоидальную функцию

In [19]:
# импортируем класс TargetEncoder из пакета category_encoders, предварительно 
# установив его с помощью команды conda install -c conda-forge category_encoders
from category_encoders import TargetEncoder

# создаем экземпляр класса TargetEncoder (модель)
# для обучающей выборки
target_enc = TargetEncoder(cols=['living_region', 'job_position'], 
                           smoothing=2, 
                           min_samples_leaf=4)

# обучаем модель, т.е. создаем таблицу, в соответствии с которой 
# категориям предиктора в обучающей выборке будут сопоставлены 
# сглаженные средние значения зависимой переменной
target_enc.fit(X_train, y_train)

# применяем модель к обучающей выборке, категории предиктора 
# в обучающей выборке заменяются на сглаженные средние значения зависимой 
# переменной
target_encoded_train = target_enc.transform(X_train)

# создаем экземпляр класса TargetEncoder (модель)
# для тестовой выборки
target_enc_test = TargetEncoder(cols=['living_region', 'job_position'], 
                                smoothing=False)

# обучаем модель, т.е. создаем таблицу, в соответствии с которой 
# категориям предиктора в тестовой выборке будут сопоставлены 
# обычные средние значения зависимой переменной в этих категориях,
# вычисленные на обучающей выборке
target_enc_test.fit(X_train, y_train)

# применяем модель к тестовому массиву признаков,
# категории предиктора в тестовой выборке заменяются на обычные 
# средние значения зависимой переменной в этих категориях,
# вычисленные на обучающей выборке
target_encoded_test = target_enc_test.transform(X_test)

In [20]:
# взглянем на результаты кодировки в обучающем массиве признаков
target_encoded_train

,living_region,job_position
0,0.425169,0.358333
1,0.425169,0.491035
2,0.674153,0.358333
3,0.425169,0.549661
4,0.674153,0.549661
5,0.674153,0.549661
6,0.674153,0.549661
7,0.674153,0.549661
8,0.425169,0.491035
9,0.425169,0.491035


In [21]:
# взглянем на результаты кодировки в тестовом массиве признаков
target_encoded_test

,living_region,job_position
0,0.8,0.25
1,0.4,0.50
2,0.8,0.25
3,0.4,0.60
4,0.8,0.25
5,0.8,0.60
6,0.8,0.60
7,0.4,0.50
8,0.4,0.50
9,0.2,0.50


### Кодирование средним значением зависимой переменной, сглаженным через сигмоидальную функцию, по схеме K-fold

In [22]:
# импортируем библиотеку h2o и
# подключаем кластер H2O
import h2o
h2o.init(nthreads=-1, max_mem_size=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_202"; Java(TM) SE Runtime Environment (build 1.8.0_202-b08); Java HotSpot(TM) 64-Bit Server VM (build 25.202-b08, mixed mode)
  Starting server from /anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmp_iy2mmxo
  JVM stdout: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmp_iy2mmxo/h2o_artemgruzdev_started_from_python.out
  JVM stderr: /var/folders/y_/s7c_myjd7qg6zs3hcfflpgwr0000gn/T/tmp_iy2mmxo/h2o_artemgruzdev_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Moscow
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.5
H2O cluster version age:,3 months and 19 days !!!
H2O cluster name:,H2O_from_python_artemgruzdev_0t5gnb
H2O cluster total nodes:,1
H2O cluster free memory:,7.111 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [23]:
# преобразовываем датафреймы во фреймы H2O
training = h2o.H2OFrame(train)
testing = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
# смотрим обучающий фрейм
training.describe()

Rows:15
Cols:3




,living_region,job_position,open_account_flg
type,enum,enum,int
mins,,,0.0
mean,,,0.4666666666666667
maxs,,,1.0
sigma,,,0.5163977794943222
zeros,,,8
missing,0,0,0
0,Московская область,Служащий,0.0
1,Московская область,Заместитель руководителя,1.0
2,Пермский край,Служащий,1.0


In [25]:
# меняем тип зависимой переменной
training['open_account_flg'] = training['open_account_flg'].asfactor()
testing['open_account_flg'] = testing['open_account_flg'].asfactor()
training.describe()

Rows:15
Cols:3




,living_region,job_position,open_account_flg
type,enum,enum,enum
mins,,,
mean,,,
maxs,,,
sigma,,,
zeros,,,
missing,0,0,0
0,Московская область,Служащий,0
1,Московская область,Заместитель руководителя,1
2,Пермский край,Служащий,1


In [26]:
# импортируем класс TargetEncoder библиотеки H2O
from h2o.targetencoder import TargetEncoder

# создаем столбец с индексами блоков
training['cv_fold_te'] = training.kfold_column(n_folds=5, seed=42)

# создаем экземпляр класса TargetEncoder (модель)
# для обучающей выборки
targetEncoder = TargetEncoder(x=['living_region'], 
                              blending_avg=True, 
                              inflection_point=3, 
                              smoothing=1,
                              y='open_account_flg', 
                              fold_column='cv_fold_te')

# обучаем модель, т.е. создаем таблицу, в соответствии с которой 
# категориям предиктора в обучающей выборке будут сопоставлены 
# сглаженные средние значения зависимой переменной
targetEncoder.fit(training)

In [27]:
# применяем модель к обучающей выборке, категории предиктора 
# в обучающей выборке заменяются на сглаженные средние 
# значения зависимой переменной
training = targetEncoder.transform(frame=training, 
                                   holdout_type='kfold',
                                   noise=0
                                  )

In [28]:
# создаем экземпляр класса TargetEncoder (модель)
# для тестовой выборки
targetEncoder_test = TargetEncoder(x=['living_region'], 
                                   y='open_account_flg', 
                                   blending_avg=False)

# обучаем модель, т.е. создаем таблицу, в соответствии с которой 
# категориям предиктора в тестовой выборке будут сопоставлены 
# обычные средние значения зависимой переменной в этих категориях,
# вычисленные на обучающей выборке
targetEncoder_test.fit(training)

In [29]:
# применяем модель к тестовой выборке,
# категории предиктора в тестовой выборке заменяются на обычные 
# средние значения зависимой переменной в этих категориях,
# вычисленные на обучающей выборке
testing = targetEncoder_test.transform(frame=testing, 
                                       holdout_type='none',
                                       noise=0
                                      )

In [30]:
# взглянем на результаты кодирования, полученные 
# для обучающего фрейма
training[['cv_fold_te', 'living_region', 
          'living_region_te', 'open_account_flg']].as_data_frame()

,cv_fold_te,living_region,living_region_te,open_account_flg
0,1,Московская область,0.400000,0
1,1,Московская область,0.400000,1
2,2,Московская область,0.400000,1
3,2,Московская область,0.400000,0
4,4,Московская область,0.491035,0
5,0,Пермский край,0.673800,1
6,1,Пермский край,0.566667,1
7,1,Пермский край,0.566667,1
8,3,Пермский край,0.673800,1
9,4,Пермский край,0.856565,0


In [31]:
testing[['living_region', 'living_region_te']].as_data_frame()

,living_region,living_region_te
0,Московская область,0.4
1,Московская область,0.4
2,Московская область,0.4
3,Московская область,0.4
4,Московская область,0.4
5,Пермский край,0.8
6,Пермский край,0.8
7,Пермский край,0.8
8,Пермский край,0.8
9,Пермский край,0.8


### Кодирование средним значением зависимой переменной, сглаженным через параметр регуляризации

In [32]:
# пишем функцию, которая выполняет кодирование средним значением 
# зависимой переменной, сглаженным через параметр регуляризации
def simple_smooth_mean(df, predictor, target, alpha):
    # вычисляем глобальное среднее
    mean = df[target].mean()

    # вычисляем частоты и средние по каждой категории
    agg = df.groupby(predictor)[target].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # вычисляем сглаженные средние
    smooth = (counts * means + alpha * mean) / (counts + alpha)

    # заменяем каждое значение соответствующим
    # сглаженным средним
    return df[predictor].map(smooth)

In [33]:
# выполняем кодирование и смотрим результаты
tr['simple_smooth_mean_encoded'] = simple_smooth_mean(tr, 
                                                      predictor='Class', 
                                                      target='Response', 
                                                      alpha=2)
tr

,Class,Response,class_labelenc,class_abs_freq,class_rel_freq,mean_encoded,simple_smooth_mean_encoded
0,A,1,0,4,0.444444,0.750000,0.759259
1,A,0,0,4,0.444444,0.750000,0.759259
2,A,1,0,4,0.444444,0.750000,0.759259
3,A,1,0,4,0.444444,0.750000,0.759259
4,B,1,1,3,0.333333,0.666667,0.711111
5,B,1,1,3,0.333333,0.666667,0.711111
6,B,0,1,3,0.333333,0.666667,0.711111
7,C,1,2,2,0.222222,1.000000,0.888889
8,C,1,2,2,0.222222,1.000000,0.888889


### Кодирование средним значением зависимой переменной, вычисленным по "прошлому" (упрощенный вариант кодировки, применяющейся в библиотеке CatBoost)

In [34]:
# удалим результаты наших экспериментов из датафрейма tr
tr.drop(['class_labelenc', 'class_abs_freq', 
         'class_rel_freq', 'mean_encoded', 
         'simple_smooth_mean_encoded'], axis=1, inplace=True)

In [35]:
# импортируем функцию shuffle() для перемешивания данных
from sklearn.utils import shuffle

In [36]:
# пишем функцию, которая выполняет кодирование средними значениями 
# зависимой переменной, вычисленными по "прошлому"
def history_mean_encoding(df, predictor, target):
    np.random.seed(6)
    shuffle(df)
    cumsum = df.groupby(predictor)[target].cumsum() - df[target]
    cumcnt = df.groupby(predictor).cumcount()
    global_mean = df[target].mean()
    df[predictor+'_history_mean_encoded'] = (cumsum / cumcnt).fillna(global_mean)
    return df

# выполняем кодирование средними значениями 
# зависимой переменной, вычисленными по "прошлому"
history_mean_encoding(tr, 'Class', 'Response')
# смотрим датафрейм
tr

,Class,Response,Class_history_mean_encoded
0,A,1,0.777778
1,A,0,1.000000
2,A,1,0.500000
3,A,1,0.666667
4,B,1,0.777778
5,B,1,1.000000
6,B,0,1.000000
7,C,1,0.777778
8,C,1,1.000000


## Присвоение категориям в зависимости от порядка их появления целочисленных значений, начиная с 1 ( Ordinal Encoding)

In [37]:
# импортируем класс OrdinalEncoder из пакета category_encoders
from category_encoders import OrdinalEncoder

# создаем экземпляр класса OrdinalEncoder
ordinal_enc = OrdinalEncoder(cols=['living_region', 'job_position']).fit(X_train, y_train)

# выполняем кодирование переменных living_region 
# и job_position в обучающем и тестовом наборах
ordinal_encoded_train = ordinal_enc.transform(X_train)

# взглянем на результаты кодировки 
# в обучающем массиве признаков
ordinal_encoded_train

,living_region,job_position
0,1,1
1,1,2
2,2,1
3,1,3
4,2,3
5,2,3
6,2,3
7,2,3
8,1,2
9,1,2


## Бинарное кодирование (Binary Encoding)

In [38]:
# импортируем класс BinaryEncoder из пакета category_encoders
from category_encoders import BinaryEncoder

# создаем экземпляр класса BinaryEncoder
binary_enc = BinaryEncoder(cols=['living_region', 'job_position']).fit(X_train, y_train)

# выполняем бинарное кодирование
binary_encoded_train = binary_enc.transform(X_train)

# взглянем на результаты кодировки 
# в обучающем массиве признаков
binary_encoded_train

,living_region_0,living_region_1,living_region_2,job_position_0,job_position_1,job_position_2
0,0,0,1,0,0,1
1,0,0,1,0,1,0
2,0,1,0,0,0,1
3,0,0,1,0,1,1
4,0,1,0,0,1,1
5,0,1,0,0,1,1
6,0,1,0,0,1,1
7,0,1,0,0,1,1
8,0,0,1,0,1,0
9,0,0,1,0,1,0


## Бинарное кодирование с хэшированием (Hashing)

In [39]:
# создадим игрушечный датафрейм
df = pd.DataFrame({'color':['a', 'b', 'a', 'c'], 'outcome':[0, 1, 0, 0]})
# выделяем массив признаков и массив меток
X = df.drop('outcome', axis=1)
y = df.drop('color', axis=1)
# смотрим датафрейм
df

,color,outcome
0,a,0
1,b,1
2,a,0
3,c,0


In [40]:
# импортируем класс HashingEncoder из пакета category_encoders
from category_encoders import HashingEncoder

# создаем экземпляр класса HashingEncoder
hash_enc = HashingEncoder(cols=['color'])

# обучаем модель (вычисляем хеш-функцию) и индексируем
hash_enc.fit_transform(X, y)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7
0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1
2,0,1,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0


## Создание переменных-взаимодействий (interactions)

In [41]:
# пишем функцию, которая создает взаимодействие 
# в результате конъюнкции переменных 
# feature1 и feature2
def make_conj(df, feature1, feature2):
    df[feature1 + ' + ' + feature2] = df[feature1].astype(str) + ' + ' + df[feature2].astype(str)
make_conj(train, 'living_region', 'job_position')

# еще можно еще так
train['liv_reg + job_pos'] = train.apply(
    lambda x: f"{x['living_region']} + {x['job_position']}", 
    axis=1)

train.head()

,living_region,job_position,open_account_flg,living_region + job_position,liv_reg + job_pos
0,Московская область,Служащий,0,Московская область + Служащий,Московская область + Служащий
1,Московская область,Заместитель руководителя,1,Московская область + Заместитель руководителя,Московская область + Заместитель руководителя
2,Пермский край,Служащий,1,Пермский край + Служащий,Пермский край + Служащий
3,Московская область,Руководитель,0,Московская область + Руководитель,Московская область + Руководитель
4,Пермский край,Руководитель,1,Пермский край + Руководитель,Пермский край + Руководитель


## Биннинг

In [42]:
# загружаем и смотрим данные
dev = pd.read_csv('Data/Stat_FE3.csv', sep=';')
dev.head()

,tariff_id,credit_sum,monthly_income,open_account_flg
0,1_4,33579.0,36000.0,0
1,1_32,23511.0,45000.0,0
2,1_5,39990.0,50000.0,0
3,1_3,3490.0,35000.0,0
4,1_6,36358.0,50000.0,0


### Биннинг на основе интервалов, созданных вручную или одинаковой ширины

In [43]:
# задаем точки, в которых будут находится границы интервалов 
# (до 50000, от 50000 до 200000, от 200000 и выше) 
bins = [-np.inf, 50000, 200000, np.inf]
# задаем метки для категорий будущей переменной
group_names = ['Low', 'Average', 'High']
# осуществляем биннинг переменной monthly_income 
# и записываем результаты в новую переменную incomecat
dev['incomecat'] = pd.cut(dev['monthly_income'], bins, 
                            labels=group_names)

# а теперь выполним биннинг на основе интервалов одинаковой ширины
dev['incomecat2'] = pd.cut(dev['monthly_income'], 10)
# взглянем на результаты
dev.head()

,tariff_id,credit_sum,monthly_income,open_account_flg,incomecat,incomecat2
0,1_4,33579.0,36000.0,0,Low,"(4105.0, 94500.0]"
1,1_32,23511.0,45000.0,0,Low,"(4105.0, 94500.0]"
2,1_5,39990.0,50000.0,0,Low,"(4105.0, 94500.0]"
3,1_3,3490.0,35000.0,0,Low,"(4105.0, 94500.0]"
4,1_6,36358.0,50000.0,0,Low,"(4105.0, 94500.0]"


In [44]:
print(dev['incomecat'].value_counts())
print('')
print(dev['incomecat2'].value_counts())

Low        96522
Average    22803
High         197
Name: incomecat, dtype: int64

(4105.0, 94500.0]       115748
(94500.0, 184000.0]       3438
(184000.0, 273500.0]       224
(273500.0, 363000.0]        77
(363000.0, 452500.0]        17
(452500.0, 542000.0]         7
(721000.0, 810500.0]         4
(542000.0, 631500.0]         4
(631500.0, 721000.0]         2
(810500.0, 900000.0]         1
Name: incomecat2, dtype: int64


### Биннинг на основе децилей

In [45]:
# осуществляем биннинг переменной monthly_income 
# на основе децилей и записываем результаты 
# в новую переменную income_decile
dev['income_decile'] = pd.qcut(dev['monthly_income'], 10)
dev.head()

,tariff_id,credit_sum,monthly_income,open_account_flg,incomecat,incomecat2,income_decile
0,1_4,33579.0,36000.0,0,Low,"(4105.0, 94500.0]","(35000.0, 40000.0]"
1,1_32,23511.0,45000.0,0,Low,"(4105.0, 94500.0]","(40000.0, 45000.0]"
2,1_5,39990.0,50000.0,0,Low,"(4105.0, 94500.0]","(45000.0, 50000.0]"
3,1_3,3490.0,35000.0,0,Low,"(4105.0, 94500.0]","(30000.0, 35000.0]"
4,1_6,36358.0,50000.0,0,Low,"(4105.0, 94500.0]","(45000.0, 50000.0]"


### Биннинг на основе WoE и IV

In [46]:
# взглянем на минимальное и максимальное значения
print(dev['credit_sum'].min())
print(dev['credit_sum'].max())

2736.0
200000.0


In [47]:
# задаем точки, в которых будут находится границы категорий 
# будущей переменной credsumcat
bins = [-np.inf, 10000, 30000, 50000, np.inf]
# осуществляем биннинг переменной credit_sum и записываем
# результаты в новую переменную credsumcat
dev['credsumcat'] = pd.cut(dev['credit_sum'], bins).astype('object')

In [48]:
# строим таблицу сопряженности credsumcat * open_account_flg
biv = pd.crosstab(dev['credsumcat'], dev['open_account_flg'])
biv

open_account_flg,0,1
credsumcat,,
"(-inf, 10000.0]",7378,2825
"(10000.0, 30000.0]",62408,13428
"(30000.0, 50000.0]",17921,3299
"(50000.0, inf]",10796,1467


In [49]:
# пишем функцию, которая вычисляет WoE для 
# каждой категории выбранной переменной, 
# добавляем a=0.0001, чтобы избежать деления на 0
def WoE(df, feature, target):
    biv = pd.crosstab(df[feature], df[target].astype('str'))
    a = 0.0001
    WoE = np.log((biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a))
    return WoE

In [50]:
# вычисляем WoE для каждой категории переменной credsumcat
WoE(dev, 'credsumcat', 'open_account_flg')

credsumcat
(-inf, 10000.0]      -0.584076
(10000.0, 30000.0]   -0.008308
(30000.0, 50000.0]    0.147606
(50000.0, inf]        0.450776
dtype: float64

In [51]:
# пишем функцию, которая вычисляет IV для каждой категории 
# выбранной переменной, добавляем a = 0.0001, чтобы 
# избежать деления на 0
def IV_cat(df, feature, target):
    biv = pd.crosstab(df[feature], df[target].astype('str'))
    a = 0.0001
    IV_cat = ((biv['0'] / sum(biv['0']) + a) - 
              (biv['1'] / sum(biv['1']) + a)) * np.log(
        (biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a))
    return IV_cat

In [52]:
# вычисляем IV для каждой категории переменной credsumcat
IV_cat(dev, 'credsumcat', 'open_account_flg')

credsumcat
(-inf, 10000.0]       0.034753
(10000.0, 30000.0]    0.000044
(30000.0, 50000.0]    0.003687
(50000.0, inf]        0.017944
dtype: float64

In [53]:
# пишем функцию, которая вычисляет итоговое IV для выбранной переменной, 
# добавляем a = 0.0001, чтобы избежать деления на 0
def IV(df, feature, target):
    biv = pd.crosstab(df[feature], df[target].astype('str'))
    a = 0.0001
    IV = sum(((biv['0'] / sum(biv['0']) + a) - 
             (biv['1'] / sum(biv['1']) + a)) * np.log(
        (biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a)))
    return IV

In [54]:
# вычисляем итоговое IV для переменной credsumcat
IV(dev, 'credsumcat', 'open_account_flg')

0.05642812895354752

In [55]:
# пишем функцию, вычисляющую IV по всем 
# количественным предикторам
def numeric_IV(df): 
    # создаем список, в который будем записывать IV
    iv_list = []
    # создаем копию датафрейма
    df = df.copy()
    # записываем константу, которую будем добавлять, чтобы избежать деления на 0
    a = 0.0001 
    # задаем зависимую переменную
    target = df['open_account_flg'].astype('str')
    # отбираем столбцы, у которых больше 10 уникальных значений
    df = df.loc[:, df.apply(pd.Series.nunique) > 10]
    # из этих столбцов отбираем только количественные
    numerical_columns = df.select_dtypes(include=['number']).columns
    # запускаем цикл, который вычисляет IV по каждой выбранной переменной
    for var_name in numerical_columns:
        # разбиваем переменную на 10 квантилей
        df[var_name] = pd.qcut(df[var_name].values, 10, duplicates='drop').codes
        # строим таблицу сопряженности между категоризированной переменной и зависимой переменной
        biv = pd.crosstab(df[var_name], target)        
        # вычисляем IV на основе таблицы сопряженности
        IV = sum(((biv['0'] / sum(biv['0']) + a) - (biv['1'] / sum(biv['1']) + a)) * np.log(
            (biv['0'] / sum(biv['0']) + a) / (biv['1'] / sum(biv['1']) + a)))
        iv_list.append(IV) # добавляем вычисленное IV в список, где хранятся IV
    col_list = list(numerical_columns) # создаем список с названиями столбцов
    # создаем датафрейм с двумя столбцами, в одном - названия переменных, в другом - IV этих переменных
    result = pd.DataFrame({'Название переменной': col_list, 'IV': iv_list})
    # добавляем дополнительный столбец "Полезность"
    result['Полезность'] = ['Подозрительно высокая' if x > 0.5 else 'Сильная' 
                            if x <= 0.5 and x > 0.3 else 'Средняя'
                            if x <= 0.3 and x > 0.1 else 'Слабая' 
                            if x <= 0.1 and x > 0.02 else 'Бесполезная' 
                            for x in result['IV']]  # по Наиму Сиддики
    # возвращаем датафрейм, отсортированный по убыванию IV
    return(result.sort_values(by='IV', ascending=False))

In [56]:
numeric_IV(dev)

,Название переменной,IV,Полезность
0,credit_sum,0.065526,Слабая
1,monthly_income,0.005984,Бесполезная


### Биннинг на основе WoE в пакете PyWoE

In [57]:
# импортируем необходимые классы
from woe import *

In [58]:
# создаем модель - экземпляр класса WoE, задаем максимально возможное 
# количество бинов, минимальное количество наблюдений в
# бине, тип предиктора, тип зависимой переменной
woe = WoE(qnt_num=10, min_block_size=10, v_type='c', t_type='b')

In [59]:
# обучаем модель - вычисляем WoE
woe.fit(dev['credit_sum'], dev['open_account_flg'])

In [60]:
# выполняем WoE-трансформацию переменной credit_sum
woe.transform(dev['credit_sum'])
# уменьшаем количество знаков после десятичной точки
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# выводим информацию о бинах
print(woe.bins)

    mean   bad    obs  good    woe      bins labels
0  0.274  2976  10871  7895 -0.569      -inf      0
1  0.168  1826  10879  9053  0.056 10290.000      1
2  0.151  1639  10849  9210  0.182 13109.000      2
3  0.177  1928  10866  8938 -0.011 15348.000      3
4  0.200  2168  10866  8698 -0.155 17284.000      4
5  0.196  2125  10864  8739 -0.131 19827.000      5
6  0.182  1974  10871  8897 -0.039 22877.100      6
7  0.162  1759  10878  9119  0.101 26369.000      7
8  0.181  1965  10846  8881 -0.036 30468.000      8
9  0.127  1380  10866  9486  0.383 40197.636      9
10 0.118  1279  10866  9587  0.470 52499.636     10


In [61]:
# улучшаем монотонность
woe_monotonic = woe.force_monotonic(hypothesis=0)
print(woe_monotonic.bins)

   mean    bad    obs   good    woe      bins labels
0 0.274   2976  10871   7895 -0.569      -inf      0
1 0.179  11660  65195  53535 -0.020 10290.000      1
2 0.171   3724  21724  18000  0.031 26369.000      2
3 0.127   1380  10866   9486  0.383 40197.636      3
4 0.118   1279  10866   9587  0.470 52499.636      4


In [62]:
# выполняем биннинг по WoE с оптимизацией по AUC
# (используется дерево CART)
woe2 = woe.optimize(max_depth=3, scoring='roc_auc', cv=5)
print(woe2.bins)

   mean    bad    obs   good    woe      bins labels
0 0.317   1460   4605   3145 -0.777      -inf      0
1 0.243   1509   6206   4697 -0.409  7284.000      1
2 0.177  15652  88557  72905 -0.006 10272.500      2
3 0.119   2398  20154  17756  0.457 42258.500      3


### Биннинг на основе хи-квадрат (первый этап метода CHAID)

In [63]:
from CHAID import Tree
# задаем название предиктора
independent_variable = 'tariff_id'
# задаем название зависимой переменной
dep_variable = 'open_account_flg'
# создаем словарь, где ключом будет название 
# предиктора, а значением - тип переменной
dct = {independent_variable: 'nominal'}
# строим дерево CHAID и выводим его
tree = Tree.from_pandas_df(dev, dct, dep_variable, max_depth=1)
tree.print_tree()

([], {0: 98503.0, 1: 21019.0}, (tariff_id, p=0.0, score=4849.043527542213, groups=[['1_0', '1_9'], ['1_1', '1_17'], ['1_16', '1_5', '1_2', '1_7', '1_94'], ['1_19', '1_4'], ['1_21', '1_22', '1_23'], ['1_24', '1_25', '1_6'], ['1_3', '1_41', '1_91'], ['1_32', '1_99'], ['1_43', '1_44']]), dof=8))
|-- (['1_0', '1_9'], {0: 5069.0, 1: 359.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_1', '1_17'], {0: 39687.0, 1: 9244.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_16', '1_5', '1_2', '1_7', '1_94'], {0: 8504.0, 1: 940.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_19', '1_4'], {0: 8060.0, 1: 1053.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_21', '1_22', '1_23'], {0: 906.0, 1: 35.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_24', '1_25', '1_6'], {0: 23847.0, 1: 3934.0}, <Invalid Chaid Split> - the max depth has been reached)
|-- (['1_3', '1_41', '1_91'], {0: 1879.0, 1: 774.0}, <Inval